In [1]:
#data preparing

In [2]:
import numpy as np 
import pandas as pd
train_data1 = pd.read_csv('../input/dataset/solubility_train.csv')
dev_data1 = pd.read_csv('../input/dataset/solubility_dev.csv')
test_data = pd.read_csv('../input/dataset/solubility_test.csv')
train_data = pd.concat([train_data1,dev_data1])
train_data.head()

,Meta,Sequence,Label
0,Psol_train_0,GMILKTNLFGHTYQFKSITDVLAKANEEKSGDRLAGVAAESAEERV...,1
1,Psol_train_1,MAHHHHHHMSFFRMKRRLNFVVKRGIEELWENSFLDNNVDMKKIEY...,0
2,Psol_train_2,MGSDKIHHHHHHMEKSIQDTIHGVIKLEDWMVEIVDTPQFQRLRRI...,0
3,Psol_train_3,MEKYIHSVEDYHRLISYLENNLNYEDSVVNHVIYVIAKTGMRYGEI...,0
4,Psol_train_4,MSLTDSFTVRSIEGVCFRYPLATPVVTSFGKMLNRPAVFVRVVDED...,0


In [3]:
! pip install pytorch_pretrained_bert
import sys
import numpy as np
import random as rn
import pandas as pd
import torch
from pytorch_pretrained_bert import BertModel
from torch import nn
from pytorch_pretrained_bert import BertTokenizer
from keras.preprocessing.sequence import pad_sequences
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from torch.optim import Adam
from torch.nn.utils import clip_grad_norm_
from IPython.display import clear_output
import matplotlib.pyplot as plt

     |████████████████████████████████| 123 kB 639 kB/s 


In [4]:
#初始化种子稳定结果
rn.seed(321)
np.random.seed(321)
torch.manual_seed(321)
torch.cuda.manual_seed(321)

In [5]:
train_data = train_data[:-1]
test_data = test_data[:-1]

train_data = train_data.to_dict(orient='records')
test_data = test_data.to_dict(orient='records')
type(train_data)

list

In [6]:
train_seq, train_labels = list(zip(*map(lambda d: (d['Sequence'], d['Label']), train_data)))
test_seq, test_labels = list(zip(*map(lambda d: (d['Sequence'], d['Label']), test_data)))

len(train_seq), len(train_labels), len(test_seq), len(test_labels)

(69419, 69419, 2000, 2000)

In [7]:
train_seq[0]

'GMILKTNLFGHTYQFKSITDVLAKANEEKSGDRLAGVAAESAEERVAAKVVLSKMTLGDLRNNPVVPYETDEVTRIIQDQVNDRIHDSIKNWTVEELREWILDHKTTDADIKRVARGLTSEIIAAVTKLMSNLDLIYGAKKIRVIAHANTTIGLPGTFSARLQPNHPTDDPDGILASLMEGLTYGIGDAVIGLNPVDDSTDSVVRLLNKFEEFRSKWDVPTQTCVLAHVKTQMEAMRRGAPTGLVFQSIAGSEKGNTAFGFDGATIEEARQLALQSGAATGPNVMYFETGQGSELSSDAHFGVDQVTMEARCYGFAKKFDPFLVNTVVGFIGPEYLYDSKQVIRAGLEDHFMGKLTGISMGCDVCYTNHMKADQNDVENLSVLLTAAGCNFIMGIPHGDDVMLNYQTTGYHETATLRELFGLKPIKEFDQWMEKMGFSENGKLTSRAGDASIFLK'

In [8]:
#Preparing Token embeddings

In [9]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

100%|██████████| 231508/231508 [00:00<00:00, 719165.91B/s]


In [10]:
train_tokens = list(map(lambda t: ['[CLS]'] + tokenizer.tokenize(t)[:510] + ['[SEP]'], train_seq))
test_tokens = list(map(lambda t: ['[CLS]'] + tokenizer.tokenize(t)[:510] + ['[SEP]'], test_seq))
len(train_tokens), len(test_tokens)

(69419, 2000)

In [11]:
#Preparing Token IDs
train_tokens_ids = pad_sequences(list(map(tokenizer.convert_tokens_to_ids, train_tokens)), maxlen=512, truncating="post", padding="post", dtype="int")
test_tokens_ids = pad_sequences(list(map(tokenizer.convert_tokens_to_ids, test_tokens)), maxlen=512, truncating="post", padding="post", dtype="int")
train_tokens_ids.shape, test_tokens_ids.shape

((69419, 512), (2000, 512))

In [12]:
train_y = np.array(train_labels) == 0
test_y = np.array(test_labels) == 0
train_y.shape, test_y.shape, np.mean(train_y), np.mean(test_y)

((69419,), (2000,), 0.5826646883418084, 0.5)

In [13]:
train_masks = [[float(i > 0) for i in ii] for ii in train_tokens_ids]
test_masks = [[float(i > 0) for i in ii] for ii in test_tokens_ids]

In [14]:
#Baseline

In [15]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.metrics import classification_report

In [16]:
from keras.callbacks import CSVLogger

csv_logger = CSVLogger('log.csv', append=True, separator=';')

In [17]:
baseline_model = make_pipeline(CountVectorizer(ngram_range=(1,3)), LogisticRegression()).fit(train_seq, train_labels)

In [18]:
baseline_predicted = baseline_model.predict(test_seq)

In [19]:
print(classification_report(test_labels, baseline_predicted))

              precision    recall  f1-score   support

           0       0.50      1.00      0.67      1000
           1       0.00      0.00      0.00      1000

    accuracy                           0.50      2000
   macro avg       0.25      0.50      0.33      2000
weighted avg       0.25      0.50      0.33      2000



/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [20]:
#BERT model

In [21]:
class BertBinaryClassifier(nn.Module):
    def __init__(self, dropout=0.1):
        super(BertBinaryClassifier, self).__init__()

        self.bert = BertModel.from_pretrained('bert-base-uncased')

        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(768, 1)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, tokens, masks=None):
        _, pooled_output = self.bert(tokens, attention_mask=masks, output_all_encoded_layers=False)
        dropout_output = self.dropout(pooled_output)
        linear_output = self.linear(dropout_output)
        proba = self.sigmoid(linear_output)
        return proba

In [22]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [23]:
str(torch.cuda.memory_allocated(device)/1000000 ) + 'M'

'0.0M'

In [24]:
bert_clf = BertBinaryClassifier()
bert_clf = bert_clf.cuda() 

100%|██████████| 407873900/407873900 [00:15<00:00, 26535927.17B/s]


In [25]:
str(torch.cuda.memory_allocated(device)/1000000 ) + 'M'

'439.065088M'

In [26]:
x = torch.tensor(train_tokens_ids[:3]).to(device)
y, pooled = bert_clf.bert(x, output_all_encoded_layers=False)
x.shape, y.shape, pooled.shape

(torch.Size([3, 512]), torch.Size([3, 512, 768]), torch.Size([3, 768]))

In [27]:
y = bert_clf(x)  
y.cpu().detach().numpy()        # kinda Garbage Collector to free up used and cache space

array([[0.44251397],
       [0.46358287],
       [0.45854843]], dtype=float32)

In [28]:
# Cross- checking CUDA GPU Memory to ensure GPU memory is not overflowing.
str(torch.cuda.memory_allocated(device)/1000000 ) + 'M'

'6697.349632M'

In [29]:
y, x, pooled = None, None, None
torch.cuda.empty_cache()     # Clearing Cache space for fresh Model run
str(torch.cuda.memory_allocated(device)/1000000 ) + 'M'

'439.065088M'

In [30]:
#Fine Tune BERT

In [31]:
# Setting hyper-parameters

BATCH_SIZE = 8
EPOCHS = 5

In [32]:
train_tokens_tensor = torch.tensor(train_tokens_ids)
train_y_tensor = torch.tensor(train_y.reshape(-1, 1)).float()

test_tokens_tensor = torch.tensor(test_tokens_ids)
test_y_tensor = torch.tensor(test_y.reshape(-1, 1)).float()

train_masks_tensor = torch.tensor(train_masks)
test_masks_tensor = torch.tensor(test_masks)

str(torch.cuda.memory_allocated(device)/1000000 ) + 'M'

'439.065088M'

In [33]:
train_dataset = TensorDataset(train_tokens_tensor, train_masks_tensor, train_y_tensor)
train_sampler = RandomSampler(train_dataset)
train_dataloader = DataLoader(train_dataset, sampler=train_sampler, batch_size=BATCH_SIZE)

test_dataset = TensorDataset(test_tokens_tensor, test_masks_tensor, test_y_tensor)
test_sampler = SequentialSampler(test_dataset)
test_dataloader = DataLoader(test_dataset, sampler=test_sampler, batch_size=BATCH_SIZE)

In [34]:
param_optimizer = list(bert_clf.sigmoid.named_parameters()) 
optimizer_grouped_parameters = [{"params": [p for n, p in param_optimizer]}]

In [35]:
optimizer = Adam(bert_clf.parameters(), lr=3e-6)

In [36]:
torch.cuda.empty_cache()

In [37]:
for epoch_num in range(EPOCHS):
    bert_clf.train()
    train_loss = 0
    for step_num, batch_data in enumerate(train_dataloader):
        token_ids, masks, labels = tuple(t.to(device) for t in batch_data)
        print(str(torch.cuda.memory_allocated(device)/1000000 ) + 'M')
        logits = bert_clf(token_ids, masks)
        
        loss_func = nn.BCELoss()

        batch_loss = loss_func(logits, labels)
        train_loss += batch_loss.item()
        
        
        bert_clf.zero_grad()
        batch_loss.backward()
        

        clip_grad_norm_(parameters=bert_clf.parameters(), max_norm=1.0)
        optimizer.step()
        
        clear_output(wait=True)
        print('Epoch: ', epoch_num + 1)
        print("\r" + "{0}/{1} loss: {2} ".format(step_num, len(train_data) / BATCH_SIZE, train_loss / (step_num + 1)))

Epoch:  5
8677/8677.375 loss: 0.6704716648332093 


In [38]:
bert_clf.eval()
bert_predicted = []
all_logits = []
with torch.no_grad():
    for step_num, batch_data in enumerate(test_dataloader):

        token_ids, masks, labels = tuple(t.to(device) for t in batch_data)

        logits = bert_clf(token_ids, masks)
        loss_func = nn.BCELoss()
        loss = loss_func(logits, labels)
        numpy_logits = logits.cpu().detach().numpy()
        
        bert_predicted += list(numpy_logits[:, 0] > 0.5)
        all_logits += list(numpy_logits[:, 0])

In [39]:
np.mean(bert_predicted)

0.979

In [40]:
print(classification_report(test_y, bert_predicted))

              precision    recall  f1-score   support

       False       0.83      0.04      0.07      1000
        True       0.51      0.99      0.67      1000

    accuracy                           0.51      2000
   macro avg       0.67      0.51      0.37      2000
weighted avg       0.67      0.51      0.37      2000

